# Accessing Saved Data

## Configuration

This code would normally go in a script automatically run at startup. The user would not have to worry about this.

In [0]:
%matplotlib notebook
import matplotlib.pyplot as plt

# Make plots live-update while scans run.
from bluesky.utils import install_nb_kicker
install_nb_kicker()

from bluesky import RunEngine
from bluesky.plans import scan, relative_scan, list_scan
from bluesky.callbacks import LiveTable, LivePlot

# Set up simulated hardware.
from bluesky.examples import motor, SynGauss
det = SynGauss('det', motor, 'motor', center=0, Imax=1,
               noise='uniform', sigma=1, noise_multiplier=0.1)

RE = RunEngine({})

# Set up a databroker.
import os
import tzlocal
import warnings
from portable_mds.sqlite.mds import MDS
from portable_fs.sqlite.fs import FileStore
import tempfile

dirname = dirname = tempfile.TemporaryDirectory().name
mds = MDS({'directory': dirname,
           'timezone': tzlocal.get_localzone().zone})
fs = FileStore({'dbpath': os.path.join(dirname, 'filestore.db')})

# Filter warnings to avoid confusing new users with red deprecation warnings.
warnings.simplefilter("ignore")

from databroker.broker import Broker
db = Broker(mds, fs)
RE.subscribe('all', db.insert);  # all data generated by RE will be saved into db

## Data Acquisition

### Execute a scan, retrieve the data as a table, and export it to CSV.

In [0]:
table = LiveTable(['det', 'motor_setpoint', 'motor'])

RE(scan([det], motor, -1, 1, 5), table)

Access the most recent run, and get the data as a table (a `pandas.DataFrame`).

In [0]:
header = db[-1]
header.table()

In [0]:
header.table().to_csv('my_data.csv')

In [0]:
!cat my_data.csv

### Use metadata to generate a nice filename

Most of the useful metadata in stored in the "Run Start document," which we can access in `header.start`. Let's see what's in there. (See [this page of the bluesky documentation](https://nsls-ii.github.io/bluesky/documents.html) for more about "documents".)

In [0]:
header.start

In [0]:
def export_csv(header):
    filename = "{plan_name}_{num_points}.csv".format(**header.start)
    header.table().to_csv(filename)
    print("Exported data to", filename)

In [0]:
export_csv(header)

Execute a new scan with a different number of points.

In [0]:
RE(scan([det], motor, -1, 1, 8), table)

In [0]:
export_csv(db[-1])

### Provide metadata and using it in the filename

In [0]:
def export_csv2(header):
    """Export to CSV. Expect header to have 'operator' and 'purpose' metadata."""
    filename = "{operator}_{plan_name}_{num_points}_{purpose}.csv".format(**header.start)
    header.table().to_csv(filename)
    print("Exported data to", filename)

In [0]:
# When RE receives extra keyword arguments it does not recognize,
# it captures them as metadata.
RE(scan([det], motor, -1, 1, 8), table, purpose='calibration', operator='Dan')

In [0]:
export_csv2(db[-1])

In [0]:
def overnight():
    "A multi-run plan. Each run gets different 'purpose' metadata."
    yield from scan([det], motor, -1, 1, 10, md={'purpose': 'calibration'})
    # open shutter or something
    yield from scan([det], motor, -1, 1, 10, md={'purpose': 'rough measurement'})
    yield from scan([det], motor, -1, 1, 100, md={'purpose': 'fine measurement'})

In [0]:
RE(overnight(), operator='Dan')
headers = db[-3:]  # grab the last three runs
for header in headers:
    export_csv2(header)

In [0]:
run_ids = RE(overnight(), operator='Dan')  # stash the unique IDs of these runs...
headers = db[run_ids]  # ... and use them to look up the data
for header in headers:
    export_csv2(header)

### Search for runs using user-specified metadata and plot search results together

In [0]:
fig, ax = plt.subplots()

for header in db(operator='Dan'):
    label = header.start.scan_id
    data = header.table()
    ax.plot('motor', 'det', label=label, data=data)

ax.legend()

In [0]:
fig, ax = plt.subplots()

for header in db(purpose='fine measurement'):
    label = header.start.scan_id
    data = header.table()
    ax.plot('motor', 'det', label=label, data=data)

ax.legend()

## Exercises

1. Experiment with writing variations of `export_csv`. Try writing one that sorts files into subdirectories based on operator name.
2. Try various search queries with `db()`.